**0. Load and Prepare Data**

In [14]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

# set visulization style
sns.set_style(style="whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')

In [15]:
# --- Ensure consistent working directory for data loading ---
# This block dynamically sets the current working directory to the Git repository root.
# This makes data paths reliable for all collaborators, regardless of where they open the notebook.

current_dir = os.getcwd()
repo_root = current_dir
while not os.path.exists(os.path.join(repo_root, '.git')):
    # Move up one directory
    parent_dir = os.path.dirname(repo_root)
    if parent_dir == repo_root: # Reached filesystem root, .git not found
        raise FileNotFoundError(
            "Could not find the .git directory. "
            "Please ensure you are running this code from within a Git repository."
        )
    repo_root = parent_dir

if os.getcwd() != repo_root:
    os.chdir(repo_root)
    print(f"Working directory set to: {os.getcwd()}") # Informative print for users


# --- Data Loading ---
# Path to the data file, relative to the repository root.
data_file_name = 'Customer_Purchasing_Behaviors.csv'
data_file_path = os.path.join('src', 'data', data_file_name)

try:
    df = pd.read_csv(data_file_path)
    print(f"Successfully loaded '{data_file_name}'.")
    #print(df.head())
except FileNotFoundError:
    print(f"Error: The file '{data_file_name}' was not found at '{data_file_path}'.")
    print("Please ensure it exists in the 'src/data/' folder relative to the repository root.")
except Exception as e:
    print(f"An error occurred during data loading: {e}")

# Create a copy for feature engineering to keep the original data safe
df_eng = df.copy()
print("Original DataFrame shape:", df_eng.shape)

Successfully loaded 'Customer_Purchasing_Behaviors.csv'.
Original DataFrame shape: (238, 7)


In [16]:
df_eng.columns # see the original columns

Index(['user_id', 'age', 'annual_income', 'purchase_amount', 'loyalty_score',
       'region', 'purchase_frequency'],
      dtype='object')

**1. Handling Categorical Features**

In [17]:
# Rationale: Group the underrepresented 'East' region to ensure model stability and prevent learning from statistical noise.
df_eng['region_grouped'] = df_eng['region'].replace({'East': 'North'})

# Convert categorical data into numerical format using One-Hot Encoding
region_dummies = pd.get_dummies(df_eng['region_grouped'], prefix='region', drop_first=False) # REMEMBER: When doing One-Hot encoding, do not use North, South and West; only use two(2) of them
df_eng = pd.concat([df_eng, region_dummies], axis=1)

print("\n--- DataFrame after handling 'region' ---")
print(df_eng[['user_id', 'region', 'region_grouped', 'region_South', 'region_West']].head())



--- DataFrame after handling 'region' ---
   user_id region region_grouped  region_South  region_West
0        1  North          North         False        False
1        2  South          South          True        False
2        3   West           West         False         True
3        4   East          North         False        False
4        5  North          North         False        False


**2. Creating Ratio-Based Features (Behavioral Insights)**

In [19]:
# Rationale: Ratios normalize for effects like purchase frequency and provide deeper behavioral context.
df_eng['spend_per_purchase'] = df_eng['purchase_amount'] / df_eng['purchase_frequency']
df_eng['spend_to_income_ratio'] = df_eng['purchase_amount'] / df_eng['annual_income']

print("\n--- Newly created ratio features ---")
print(df_eng[['user_id', 'spend_per_purchase', 'spend_to_income_ratio']].head())



--- Newly created ratio features ---
   user_id  spend_per_purchase  spend_to_income_ratio
0        1           16.666667               0.004444
1        2           19.444444               0.006364
2        3           22.727273               0.007692
3        4           15.000000               0.005000
4        5           16.923077               0.004681


**3. Creating Demographic Tiers (Binning)**

In [21]:
# Rationale: Converts continuous variables into interpretable categories for business analysis and segmentation.
age_bins = [14, 30, 45, 60, 100]
age_labels = ['Young_Adult', 'Adult', 'Middle_Aged', 'Senior']
df_eng['age_group'] = pd.cut(df_eng['age'], bins=age_bins, labels=age_labels, right=False)

income_bins = [0, 45000, 90000, 150000]
income_labels = ['Low_Income', 'Medium_Income', 'High_Income']
df_eng['income_bracket'] = pd.cut(df_eng['annual_income'], bins=income_bins, labels=income_labels, right=False)

print("\n--- Newly created demographic tiers ---")
print(df_eng[['user_id', 'age', 'age_group', 'annual_income', 'income_bracket']].head())


--- Newly created demographic tiers ---
   user_id  age    age_group  annual_income income_bracket
0        1   25  Young_Adult          45000  Medium_Income
1        2   34        Adult          55000  Medium_Income
2        3   45  Middle_Aged          65000  Medium_Income
3        4   22  Young_Adult          30000     Low_Income
4        5   29  Young_Adult          47000  Medium_Income


**4. Creating Composite Scores**

In [22]:
# Rationale: Combines multiple collinear features into single, powerful, and interpretable scores for value and risk.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_eng[['purchase_amount', 'purchase_frequency', 'loyalty_score']])
df_scaled = pd.DataFrame(scaled_features, columns=['purchase_scaled', 'frequency_scaled', 'loyalty_scaled'])

# Customer Value Score (weighted sum of key metrics)
weights = {'monetary': 0.5, 'frequency': 0.25, 'loyalty': 0.25}
df_eng['customer_value_score'] = (weights['monetary'] * df_scaled['purchase_scaled'] +
                                  weights['frequency'] * df_scaled['frequency_scaled'] +
                                  weights['loyalty'] * df_scaled['loyalty_scaled'])

# Churn Risk Score (high for low loyalty and frequency)
df_eng['churn_risk_score'] = (0.5 * (1 - df_scaled['loyalty_scaled']) +
                              0.5 * (1 - df_scaled['frequency_scaled']))

print("\n--- Newly created composite scores ---")
print(df_eng[['user_id', 'customer_value_score', 'churn_risk_score']].head())



--- Newly created composite scores ---
   user_id  customer_value_score  churn_risk_score
0        1              0.136490          0.829060
1        2              0.469039          0.470085
2        3              0.716117          0.282051
3        4              0.000000          1.000000
4        5              0.182326          0.778205


**5. Creating Interaction and Segmentation Features**

In [23]:
# Rationale: Create binary flags for easy filtering and to identify high-value customer segments like 'Champions'.

# Binary Segmentation Flags (based on top 25% percentile)
high_value_threshold = df_eng['purchase_amount'].quantile(0.75)
high_loyalty_threshold = df_eng['loyalty_score'].quantile(0.75)
high_frequency_threshold = df_eng['purchase_frequency'].quantile(0.75)

df_eng['is_high_value'] = (df_eng['purchase_amount'] > high_value_threshold)#.astype(int)
df_eng['is_loyal'] = (df_eng['loyalty_score'] > high_loyalty_threshold)#.astype(int)
df_eng['is_frequent'] = (df_eng['purchase_frequency'] > high_frequency_threshold)#.astype(int)
df_eng['is_champion'] = (df_eng['is_high_value'] * df_eng['is_loyal'] * df_eng['is_frequent'])#.astype(int)

print("\n--- Binary Segmentation Flags ---")
print(df_eng[['user_id', 'is_high_value', 'is_loyal', 'is_frequent', 'is_champion']].head())
print(f"Number of Champion Customers: {df_eng['is_champion'].sum()}")



--- Binary Segmentation Flags ---
   user_id  is_high_value  is_loyal  is_frequent  is_champion
0        1          False     False        False        False
1        2          False     False        False        False
2        3          False     False        False        False
3        4          False     False        False        False
4        5          False     False        False        False
Number of Champion Customers: 59


**6. Creating Statistical and Business-Savvy Features**

In [31]:
# Rationale: Create normalized ranks and business-oriented scores like 'Growth Potential'.
df_eng = df_eng.reset_index(drop=True) # this ensures the index is a simple range and matches the Series returned by .rank().
# Percentile Ranks
df_eng['income_percentile'] = df_eng['annual_income'].rank(pct=True)
df_eng['spending_percentile'] = df_eng['purchase_amount'].rank(pct=True)

# Growth Potential Score (High Income, Relatively Low Spending)
df_eng['growth_potential_score'] = df_eng['income_percentile'] - df_eng['spending_percentile']

# Scaling to range [0-100] # NOTE TO BE DELETED: Added after the meeting
scaler = MinMaxScaler(feature_range=(0, 100))
# Apply scaling
df_eng['growth_potential_score'] = scaler.fit_transform(df_eng[['growth_potential_score']]).astype(int)

print("\n--- Growth Potential & Percentile Scores ---")
print(df_eng.sort_values('growth_potential_score', ascending=False)[['user_id', 'annual_income', 'purchase_amount', 'growth_potential_score']].head())


--- Growth Potential & Percentile Scores ---
    user_id  annual_income  purchase_amount  growth_potential_score
4         5          47000              220                     100
30       31          55000              350                     100
1         2          55000              350                     100
45       46          47000              220                     100
22       23          63000              460                      88


**7. Finalizing the Model-Ready DataFrame**

In [37]:
# Rationale: Create a final, clean DataFrame containing only the identifier and the best engineered features for modeling.
# This prevents data leakage and removes redundant columns.

features_for_modeling = [
    'user_id',
    'age',
    'annual_income',
    'purchase_amount',
    'loyalty_score',
    'region',
    'region_grouped',
    'purchase_frequency',
    # --- Core Scores ---
    'customer_value_score',
    'churn_risk_score',
    'growth_potential_score',
    # --- Behavioral Ratios ---
    'spend_per_purchase',
    'spend_to_income_ratio',
    # --- Key Segments/Flags ---
    'is_high_value',
    'is_loyal',
    'is_frequent',
    'is_champion',
    # --- Demographics and income bracket---
    'age_group',
    'income_bracket',
    # --- Added these missing columns ---
    'income_percentile',
    'spending_percentile'
]


# Dynamically add the one-hot encoded region columns to the list
final_feature_list = features_for_modeling + list(region_dummies.columns)

df_model_ready = df_eng[final_feature_list].copy()

print("\n--- FINAL MODEL-READY DATAFRAME ---")
print("Shape:", df_model_ready.shape)
print("Columns:", df_model_ready.columns.tolist())
print(df_model_ready.head())



--- FINAL MODEL-READY DATAFRAME ---
Shape: (238, 24)
Columns: ['user_id', 'age', 'annual_income', 'purchase_amount', 'loyalty_score', 'region', 'region_grouped', 'purchase_frequency', 'customer_value_score', 'churn_risk_score', 'growth_potential_score', 'spend_per_purchase', 'spend_to_income_ratio', 'is_high_value', 'is_loyal', 'is_frequent', 'is_champion', 'age_group', 'income_bracket', 'income_percentile', 'spending_percentile', 'region_North', 'region_South', 'region_West']
   user_id  age  annual_income  purchase_amount  loyalty_score region  \
0        1   25          45000              200            4.5  North   
1        2   34          55000              350            7.0  South   
2        3   45          65000              500            8.0   West   
3        4   22          30000              150            3.0   East   
4        5   29          47000              220            4.8  North   

  region_grouped  purchase_frequency  customer_value_score  churn_risk_score  

**The following is an analysis of the post that Vinod posted in Slack group chat:**

**7. Segmentation features - (is_high_value, is_loyal, is_frequent, customer_tier)**

In [ ]:
# customer_tier. Would this be market segmentations? Would it come out from clustering?

**8. Demographic behavioural interaction - (young_high_spender, senior_loyal, income_age_segment, etc..)**

In [ ]:
# Demographic behavioural interaction. Would this be market segmentations? Would it come out from clustering?

**9. Statistical Features - (frequency_percentile, is_outlier_spender, loyalty_deviation, etc..)**

In [ ]:
# NOTE TO BE DELETED: Added after the meeting
# Define labels for each quantile
labels = ['0-25%', '25-50%', '50-75%', '75-100%']
# frequency_percentile
df['frequency_percentile'] = pd.qcut(df['purchase_frequency'], q=4, labels=labels)

df['frequency_percentile'].head()


In [44]:
# Exporting the feature engineered data to a csv file
model_ready_data = pd.DataFrame(df_model_ready, columns=df_eng.columns)
model_ready_data

,user_id,age,annual_income,purchase_amount,loyalty_score,region,purchase_frequency,region_grouped,region_North,region_South,...,income_bracket,customer_value_score,churn_risk_score,is_high_value,is_loyal,is_frequent,is_champion,income_percentile,spending_percentile,growth_potential_score
0,1,25,45000,200,4.5,North,12,North,True,False,...,Medium_Income,0.136490,0.829060,False,False,False,False,0.140756,0.073529,74
1,2,34,55000,350,7.0,South,18,South,False,True,...,Medium_Income,0.469039,0.470085,False,False,False,False,0.418067,0.292017,100
2,3,45,65000,500,8.0,West,22,West,False,False,...,Medium_Income,0.716117,0.282051,False,False,False,False,0.699580,0.670168,57
3,4,22,30000,150,3.0,East,10,North,True,False,...,Low_Income,0.000000,1.000000,False,False,False,False,0.004202,0.004202,44
4,5,29,47000,220,4.8,North,13,North,True,False,...,Medium_Income,0.182326,0.778205,False,False,False,False,0.207983,0.081933,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,234,40,60000,450,7.2,West,20,West,False,False,...,Medium_Income,0.606550,0.399145,False,False,False,False,0.544118,0.544118,44
234,235,38,59000,430,6.9,North,20,North,True,False,...,Medium_Income,0.574603,0.422222,False,False,False,False,0.510504,0.476891,59
235,236,54,74000,630,9.4,South,27,South,False,True,...,Medium_Income,0.972061,0.035470,True,True,True,True,0.957983,0.957983,44
236,237,32,52000,360,5.8,West,18,West,False,False,...,Medium_Income,0.433089,0.562393,False,False,False,False,0.304622,0.313025,40


In [46]:
# Export to your data folder
output_path = os.path.join('src', 'data', 'Customer_Purchasing_Behaviors_Engineered.csv')
# Export with 4 decimal places
model_ready_data.to_csv(
    output_path,
    index=False,
    float_format='%.4f'  # This formats all float columns to 4 decimal places
)
print("Data exported with 4 decimal places")

Data exported with 4 decimal places
